In [1]:
import sys
import time
import zmq
from PyQt5 import sip
import pandas as pd


from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QSize, Qt
from PyQt5.QtWidgets import *

# 全局变量

In [2]:
AcctNameDict = {}

In [3]:
priceType ={}
priceType['1'] = "AnyPrice"
priceType['2'] = "LimitPrice"
priceType['u'] = "Unknown"
priceType

{'1': 'AnyPrice', '2': 'LimitPrice', 'u': 'Unknown'}

In [4]:
directionType ={}
directionType['0'] = "Buy"
directionType['1'] = "Sell"
directionType['u'] = "Unknown"
directionType

{'0': 'Buy', '1': 'Sell', 'u': 'Unknown'}

In [5]:
OpenCloseFlag ={}
OpenCloseFlag['0'] = "Open"
OpenCloseFlag['1'] = "Close"
OpenCloseFlag['3'] = "CloseToday"
OpenCloseFlag['4'] = "CloseYesterday"
OpenCloseFlag['u'] = "Unknown"
OpenCloseFlag

{'0': 'Open',
 '1': 'Close',
 '3': 'CloseToday',
 '4': 'CloseYesterday',
 'u': 'Unknown'}

In [6]:
hedgeFlag = {}
hedgeFlag['u'] = "Unknown"
hedgeFlag['1'] = "Speculate"
hedgeFlag['2'] = "Arbitrage"
hedgeFlag

{'u': 'Unknown', '1': 'Speculate', '2': 'Arbitrage'}

In [7]:
orderStatus ={}
orderStatus['1'] = "New"
orderStatus['2'] = "Accepted"
orderStatus['3'] = "Rejected"
orderStatus['4'] = "Cancelling"
orderStatus['5'] = "Canceled"
orderStatus['6'] = "PartTradedQueueing"
orderStatus['7'] = "PartTradedNotQueueing"
orderStatus['8'] = "AllTraded"
orderStatus['u'] = "Unknown"
orderStatus

{'1': 'New',
 '2': 'Accepted',
 '3': 'Rejected',
 '4': 'Cancelling',
 '5': 'Canceled',
 '6': 'PartTradedQueueing',
 '7': 'PartTradedNotQueueing',
 '8': 'AllTraded',
 'u': 'Unknown'}

# 相关函数

获取账户 字典

In [8]:
def GetAcctName():
    tb = pd.read_csv("./depend/accountMap.txt",header=None)
    tb.index = tb[0]
    tb = tb[[1]]
    tb.index.name=None
    tb_dict = tb.to_dict()
    tb_dict = tb_dict[1]
    
    return tb_dict
GetAcctName()

{8001: 'TEST-FUTURE1',
 8002: 'TEST-STOCK1',
 8003: 'ALPHA-1-FUT',
 8004: 'ALPHA-1-STK',
 8005: 'ALPHA-2-FUT',
 8006: 'ALPHA-2-STK',
 8007: 'ALPHA-3-FUT',
 8008: 'ALPHA-3-STK',
 8009: 'ALPHA-5-FUT',
 8011: 'TEST-FUTURE2',
 8012: 'TEST-FUTURE3',
 8010: 'ALPHA-SW1-STK',
 8013: 'ALPHA_SW1_FUT',
 8014: 'JJ_STOCK',
 8015: 'SW_FUT_2',
 8016: 'JR7_STOCK',
 8017: 'SW_CREDIT_1'}

# 相关类

##  Request 类

In [9]:
###Cedar Request
class Request(object):
    rqAcct=0;
    rqInstrument="";
    rqBatchID="";
    rqid=0;
    rqDirection='u';
    rqOrderSize=0;
    tradedVol=0;
    tradedAvgPrice=0;
    fillRate=0;
    referencePrice=0;

    def fill_value(msg):
        elems = msg.split(',')
        rqAcct = int(elems[0])
        rqInstrument=elems[1]
        rqBatchID=elems[2]
        rqid=int(elems[3])
        rqDirection=elems[4]
        rqOrderSize=int(elems[5])
        tradedVol=int(elems[6])
        tradedAvgPrice=float(elems[7])
        fillRate=float(elems[8])
        referencePrice=float(elems[9])



## Order 类


In [10]:
###Cedar Order
class Order(object):
        m_OrderRef=0;           #### Order ref no, unique for each order in exchange
        m_RequestID=0;          ##/< Request id, unique for each strategy
        m_OrderSysID="";        ##/< Order sys id, used by CTP, according to CTP documentation, It's quicker to use this id when cancelling an order
        m_ProdID="";            ##/< Product id
        m_ExchID="";            ##/< Exchange id
        m_PriceType='u';        ##/< Price type, refer to \ref PriceTypeGroup
        m_Direction='u';        ##/< Order direction, refer to \ref DirectionGroup
        m_CombOffsetFlag='u';   ##/< Offest flag, refer to \ref OffsetFlagGroup
        m_CombHedgeFlag='u';    ##/< Hedge flag, refer to \ref HedgeFlagGroup
        m_LimitPrice=0;         ##/< Limit price
        m_VolOriginal=0;        ##/< Original volume of this order
        m_VolRemaining=0;       ##/< Remaining volume of this order
        m_VolTraded=0;          ##/< Traded volume of this order, updated by OnRtnOrder
        m_VolConfirmTraded=0;	##/< Confirmed traded volume of this order, updated by OnRtnTrade
        m_Status='u';           ##/< Order status, refer to \ref OrderStatusGroup
        m_ChaseTimes=0;         ##/< The times of chasing this order
        m_SequenceNo=0;         ##/< Order sequence no, updated by exchange

        def fill_value(msg):
            elems = msg.split(',')
            m_OrderRef=int(elems[0])
            m_RequestID=int(elems[1])
            m_OrderSysID=elems[2]
            m_ProdID=elems[3]
            m_ExchID=elems[4]
            m_PriceType=elems[5]
            m_Direction=elems[6]
            m_CombOffsetFlag=elems[7]
            m_CombHedgeFlag=elems[8]
            m_LimitPrice=float(elems[9])
            m_VolOriginal=int(elems[10])
            m_VolRemaining=int(elems[11])
            m_VolTraded=int(elems[12])
            m_VolConfirmTraded=int(elems[13])
            m_Status=elems[14]
            m_ChaseTimes=int(elems[15])
            m_SequenceNo=int(elems[16])



## BatchManaer类

In [11]:
class BatchManager():

    def __init__(self):
        self.BuyValuePerRQID = {}
        self.SellValuePerRQID = {}
        self.BuyTotalValuePerRQID = {}
        self.SellTotalValuePerRQID = {}
        self.AcctID = 0
    def bookAcctID(self,acct):
        self.AcctID = acct

    def bookTradedValue(self,RQID,value):
        if(value > 0):
            self.BuyValuePerRQID[RQID] = value
            #print("BUY: %f"%(self.BuyValuePerRQID[RQID]))
        else:
            self.SellValuePerRQID[RQID] = value
            #print("SELL: %f"%(self.SellValuePerRQID[RQID]))
    def bookTotalValue(self,RQID,value):
        if(value > 0):
            self.BuyTotalValuePerRQID[RQID] = value
            #print("BUY: %f"%(self.BuyValuePerRQID[RQID]))
        else:
            self.SellTotalValuePerRQID[RQID] = value
            #print("SELL: %f"%(self.SellValuePerRQID[RQID]))

    def getBuyNotional(self):
        #print("BUY NOTIONAL : ",self.BuyValuePerRQID.values(),sum(self.BuyValuePerRQID.values()))
        boughtNotional = sum(self.BuyValuePerRQID.values())
        totalBuyNotional  = sum(self.BuyTotalValuePerRQID.values())
        myFillRate     = (boughtNotional / totalBuyNotional) if (totalBuyNotional > 0) else 0
        return [boughtNotional,myFillRate,totalBuyNotional]

    def getSellNotional(self):
        #print("SELL NOTIONAL : ",self.SellValuePerRQID.values(),sum(self.SellValuePerRQID.values()))
        soldNoitional = sum(self.SellValuePerRQID.values())
        totalSellNotional = sum(self.SellTotalValuePerRQID.values())
        myFillRate    = soldNoitional / totalSellNotional if((abs(totalSellNotional)) > 0) else 0
        return [soldNoitional,myFillRate,totalSellNotional]

    def getAcctID(self):
        return self.AcctID;

## updata类

In [12]:
class UpdateData_tab3(QtCore.QThread):
    requestChanged = QtCore.pyqtSignal(int, int, list)  # rowIndex, msgType,msg list

    def run(self):
        requestChanged = QtCore.pyqtSignal(int, int, list)
        context = zmq.Context()
        sock = context.socket(zmq.SUB)
        sock.setsockopt(zmq.SUBSCRIBE, b"IND_CONTRIBUTE")
        sock.setsockopt(zmq.SUBSCRIBE, b"TOP_WINNER")
        sock.setsockopt(zmq.SUBSCRIBE, b"TOP_LOSSER")
        sock.setsockopt(zmq.HEARTBEAT_IVL,     5000)
        sock.setsockopt(zmq.HEARTBEAT_TIMEOUT, 3000)
        #print("hello")
        sock.connect("tcp://192.168.0.32:19006")
		### DOUBLE RQID Problem

        while True:
            msg = sock.recv()
            msgs = msg.decode("utf-8").split(",")
            #msgs= msg.split(',')
            #self.dataChanged.emit(2, 2, msgs[0])
            #print(msgs)
            if(msgs[0] == "IND_CONTRIBUTE"):
                self.requestChanged.emit(1,1, msgs[1:])

            elif(msgs[0] == "TOP_WINNER"):
                self.requestChanged.emit(2,2, msgs[1:])

            elif(msgs[0] == "TOP_LOSSER"):
                self.requestChanged.emit(2,3, msgs[1:])


## Control_sys_Tab

In [13]:
class Control_sys_Tab(QTabWidget):
    def __init__(self,parent=None):
        self.RequestRowKey ={}
        self.OrderRowKey  ={}
        self.BatchRowKey  ={}   #To manage batch row index
        self.ErrorRowKey  ={}   #To manage error row index
        self.BuyBatchValue = {}
        self.SellBatchValue= {}
        self.BatchManagers  ={} # To manage value for each batch
        self.g_CurrRequestRow = 0
        self.g_CurrOrderRow  = 0
        self.g_CurrBatchRow  = 0
        self.g_CurrErrorRow  = 0
        super().__init__(parent)
        
        self.setObjectName("Control_system")
        self.resize(1800,985)
        self.setWindowTitle("实时监控系统")
        
        
        #创建3个选项卡小控件窗口
        self.tab1=QWidget()
        self.tab2=QWidget()
        self.tab3=QWidget()
        
        
        #将三个选项卡添加到顶层窗口中
        self.addTab(self.tab1, "交易界面")
        self.addTab(self.tab2, "PNL展示界面")
        self.addTab(self.tab3, "PNL定时发送")

        #每个选项卡自定义的内容
        self.tab1UI()
        self.tab2UI()
        self.tab3UI()
        
        
    
    def tab1UI(self):
        
        return
        
        
    def tab2UI(self):
        
        return 
    
    def tab3UI(self):
        
        #设置主布局
        layout=QHBoxLayout()
        
        sec_layout=QFormLayout()
        
        # 创建表格窗口1
        self.tableWidget4 = QtWidgets.QTableWidget()
        self.tableWidget4.setRowCount(90)
        self.tableWidget4.setColumnCount(4)
        self.tableWidget4.setObjectName("tableWidget4")
        
        self.tableWidget4.setHorizontalHeaderLabels(["Strategy","Cat_idx","Cat_name","PNL"])
        for i in range(0,4):
            self.tableWidget4.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            self.tableWidget4.setColumnWidth(i,200)
        
        # 表格窗口2
        self.tableWidget5 = QtWidgets.QTableWidget()
        self.tableWidget5.setRowCount(3200)
        self.tableWidget5.setColumnCount(3)
        self.tableWidget5.setObjectName("tableWidget5")
        self.tableWidget5.setAutoFillBackground(True)
        self.tableWidget5.setHorizontalHeaderLabels(["Strategy", "Ticker", "PNL"])
        for i in range(0,3):
            self.tableWidget5.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            self.tableWidget5.setColumnWidth(i,240)
        
        # 表格窗口3
        self.tableWidget6 = QtWidgets.QTableWidget()

        self.tableWidget6.setRowCount(3200)
        self.tableWidget6.setColumnCount(3)

        self.tableWidget6.setObjectName("tableWidget6")
        self.tableWidget6.setAutoFillBackground(True)
        self.tableWidget6.setHorizontalHeaderLabels(["Strategy", "Ticker","PNL"])
        for i in range(0,3):
            self.tableWidget6.horizontalHeaderItem(i).setTextAlignment(Qt.AlignHCenter)
            self.tableWidget6.setColumnWidth(i,240)


        self.pushButton3 = QtWidgets.QPushButton()
        #self.pushButton.setMaximumWidth(100)
        self.pushButton3.setObjectName("pushButton3")
        self.pushButton3.setText("开始运行")
        
        
        
        # 添加表单2进 子布局
        sec_layout.addWidget(self.tableWidget5)
        
        # 添加表单3进 子布局
        sec_layout.addWidget(self.tableWidget6)
        
        # 添加 按钮 进 子布局
        sec_layout.addWidget(self.pushButton3)
        
        # 添加表单1 进主布局，子布局进主布局
        layout.addWidget(self.tableWidget4)
        layout.addLayout(sec_layout)
        

        self.tab3.setLayout(layout)
        self.pushButton3.clicked.connect(self.slotStart_tab3)
    
    ##############################################################
#     PyQt5 中的pyQtslot 是python中的decorator，用其可以将一个method 定义为 槽
    
#     槽的传参方式 主要是直接传入一个 函数指针
    
    ##############################################################
    
    
    @QtCore.pyqtSlot()
    def slotStart_tab3(self):
        #按钮 暂停使用
        self.pushButton3.setEnabled(False) 
        # 开启一个新进程用来 更新数据
        self.update_data_thread3 = UpdateData_tab3(self) 
        self.update_data_thread3.requestChanged.connect(self.onRequestChanged_tab3)
        # 线程进入 准备阶段
        self.update_data_thread3.start()

    @QtCore.pyqtSlot(int, int, list)
    def onRequestChanged_tab3(self, row, msgType, text):
        # text 即为我们所需要的数据列
        print(text)
        
        column=0
        if (msgType==1):
            
            print(msgType)
        
        
        

# main

In [14]:
if __name__ == "__main__":
    AcctNameDict = GetAcctName()#pd.read_csv("./depend/accountMap.txt")
    AcctNameDict[0] = "UNKNOWN"
    print(AcctNameDict.keys())
    app = QtWidgets.QApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)
    w = Control_sys_Tab()
    w.show()
    sys.exit(app.exec_())

dict_keys([8001, 8002, 8003, 8004, 8005, 8006, 8007, 8008, 8009, 8011, 8012, 8010, 8013, 8014, 8015, 8016, 8017, 0])
['FLML', '18', '有色金属', '-10.319']
1
['FLML', '4', '商贸零售', '-8.741']
1
['FLML', '25', '国防军工', '-7.979']
1
['FLML', '11', '交通运输', '-7.037']
1
['FLML', '2', '医药', '-6.368']
1
['FLML', '21', '计算机', '-5.273']
1
['FLML', '12', '汽车', '-4.879']
1
['FLML', '1', '房地产', '-3.556']
1
['FLML', '31', '电力设备及新能源', '-2.588']
1
['FLML', '13', '轻工制造', '-2.247']
1
['FLML', '7', '家电', '-2.230']
1
['FLML', '17', '石油石化', '-1.830']
1
['FLML', '6', '建材', '-1.775']
1
['FLML', '14', '电力及公用事业', '-1.597']
1
['FLML', '8', '纺织服装', '-0.625']
1
['FLML', '0', '银行', '-0.545']
1
['FLML', '27', '钢铁', '0.495']
1
['FLML', '32', '综合金融', '0.554']
1
['FLML', '23', '煤炭', '0.816']
1
['FLML', '22', '基础化工', '1.150']
1
['FLML', '9', '食品饮料', '3.537']
1
['FLML', '15', '综合', '3.641']
1
['FLML', '16', '通信', '4.166']
1
['FLML', '20', '建筑', '6.088']
1
['FLML', '30', '消费者服务', '6.307']
1
['FLML', '29', '电子', '6.924']
1
['FLML

SystemExit: 0

C:\Users\taosu\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
